# Assignment Selenium And Pandas

## ||| WEB SCRAPPING AND CREATING A BOT WHICH RECOMMENDS |||

##### IMPORT LIBRARIES

In [574]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import numpy as np

##### PREPARE SELENIUM TO SCRAP

In [575]:
PATH ='C:\Program Files (x86)\chromedriver.exe'

In [606]:
driver =webdriver.Chrome(PATH)

driver.get('http://www.n11.com') # I used n11.com to scrap

search = driver.find_element_by_id('searchData')

telefon_model=input('Please enter brand and model: ')

print("The Model that you have selected is :",telefon_model)
search.send_keys(telefon_model)  # Search key is Iphone 6S

search.send_keys(Keys.RETURN)

# I want to select only phones so the bot clicked to 'Cep Telefonu' button in the page.
driver.find_element_by_partial_link_text('Cep Telefonu').click()



# To sub tags, I defined main variables.

main= driver.find_element_by_id('view')
prods_tab=main.find_element_by_xpath("//ul[@class='clearfix']")
prods_list=prods_tab.find_elements_by_xpath("//li[@class='column ']")

# Then created some lists,when I scrapp I will storge each data column in each list.

prod_isim =[]
prod_yeni_fiyat=[]
prod_eski_fiyat=[]
prod_seller=[]
prod_seller_pts=[]

# Then I started to loop to scrap

for i,prods in enumerate(prods_list):
# My inital variables from website; product name,price 
    prod_name=prods.find_element_by_class_name('productName ')
    prod_detail=prods.find_element_by_class_name('proDetail')
    price=prod_detail.find_element_by_class_name('newPrice')

# In order to reach discount ratio, I decided to use find the ratio between old price and new price
# I checked that the product has got discount to determine the model
# The division between old price and new price was more suitable and progressive for the finding discount ratio,
# in HTML tags there were problems on the ratio that website already calculated so,my method would be more effective

    try:
        old_price=prod_detail.find_element_by_class_name('oldPrice')
    except NoSuchElementException:
        prod_eski_fiyat.append(price.text)
    else:
        prod_eski_fiyat.append(old_price.text)

# I defined the saller name         
    saller_detail=prods.find_element_by_class_name('sallerInfo')
    saller_name=saller_detail.find_element_by_class_name('sallerName')

# There were some seller which has no points.So I need to check wheater they have.Then define points variable.    
    try:
        saller_point=saller_detail.find_element_by_class_name('shopPoint')
    except NoSuchElementException:
        prod_seller_pts.append(0)
    else:
        prod_seller_pts.append(saller_point.text)
        
        
# Finally I got many variables from list than I append them into related lists(The other appending operations in try except points.).   
    prod_isim.append(prod_name.text)
    prod_yeni_fiyat.append(price.text)
    prod_seller.append(saller_name.text)
    
    print("*"*5,i+1," record listed","*"*5)
    
driver.close()

Please enter brand and model:  Iphone 6S


The Model that you have selected is : Iphone 6S
***** 1  record listed *****
***** 2  record listed *****
***** 3  record listed *****
***** 4  record listed *****
***** 5  record listed *****
***** 6  record listed *****
***** 7  record listed *****
***** 8  record listed *****
***** 9  record listed *****


In [607]:
# I have put my lists in Data frame, to insert it correctly I needed to transpose the list after I insert.

df=pd.DataFrame([prod_isim,prod_yeni_fiyat,prod_eski_fiyat,prod_seller,prod_seller_pts])
df=df.T
df.columns=['urun_ismi','urun_yeni_fiyat','urun_eski_fiyat','urun_satici','satici_pts']

# I write my dataframe to .csv file
df.to_csv('kronnika.csv')

## USING THE DATA

In [608]:
# From here on datas will be scraping from csv file rapidly.
df=pd.read_csv('kronnika.csv')
df=df.drop('Unnamed: 0',axis=1)
df.head()

,urun_ismi,urun_yeni_fiyat,urun_eski_fiyat,urun_satici,satici_pts
0,Apple iPhone 6S 16 GB (Apple Türkiye Garantili),"3.099,00 TL","3.499,00 TL",Azur,%95
1,Apple iPhone 6S 16 GB (İthalatçı Garantili),"1.699,00 TL","1.699,00 TL",starmobile,%100
2,Apple iPhone 6S Plus 32 GB (Apple Türkiye Gara...,"5.398,00 TL","5.500,00 TL",akay-bilişim,%100
3,Apple iPhone 6S 32 GB (Apple Türkiye Garantili),"4.199,00 TL","5.000,00 TL",akay-bilişim,%100
4,Apple iPhone 6S 32 GB (Apple Türkiye Garantili),"3.499,00 TL","3.499,00 TL",İdeaLMobiL,%100


In [609]:
# Because of Turkish Financial notaiton I need to make some adjustments on values.

def duzen(x):
    x=x.replace('.','')
    x=x.replace(',','.')
    x=x.replace(' TL','')
    x=float(x)
    return x

In [610]:
# I overwrote them to the related columns in DataFrame.

df['urun_yeni_fiyat']=list(map(duzen,df['urun_yeni_fiyat']))
df['urun_eski_fiyat']=list(map(duzen,df['urun_eski_fiyat']))
df['satici_pts']=df['satici_pts'].apply(lambda x: x.replace('%',''))
df['satici_pts']=df['satici_pts'].apply(lambda var: float(var))

In [611]:
# As I mentioned below, I got the discount ratio from old and new price.
# If there is no discount,old prices will be same as new so the discount will be 0.

df['indirim_orani']=(df['urun_eski_fiyat']/df['urun_yeni_fiyat']-1)*100
df['indirim_orani']=df['indirim_orani'].round()

In [612]:
df.head()

,urun_ismi,urun_yeni_fiyat,urun_eski_fiyat,urun_satici,satici_pts,indirim_orani
0,Apple iPhone 6S 16 GB (Apple Türkiye Garantili),3099.0,3499.0,Azur,95.0,13.0
1,Apple iPhone 6S 16 GB (İthalatçı Garantili),1699.0,1699.0,starmobile,100.0,0.0
2,Apple iPhone 6S Plus 32 GB (Apple Türkiye Gara...,5398.0,5500.0,akay-bilişim,100.0,2.0
3,Apple iPhone 6S 32 GB (Apple Türkiye Garantili),4199.0,5000.0,akay-bilişim,100.0,19.0
4,Apple iPhone 6S 32 GB (Apple Türkiye Garantili),3499.0,3499.0,İdeaLMobiL,100.0,0.0


In [613]:
# To create recommendation system I sorted the values.

df=df.sort_values(by=['urun_yeni_fiyat'],ascending=False)
df=df.reset_index(drop=True)

In [614]:
# I have calculated the absolute difference between each element in list.
# Then I found the minimum values so It gave me the closest values for each element then I wrote it in a list. 

df['fark']=list(map(lambda x: min(filter(lambda p: p>0,abs(x-df['urun_yeni_fiyat']))) ,df['urun_yeni_fiyat']))

In [615]:
# To implement recommendation I have checked the closest values list that I mentioned below which are more than 400 tl or not
# and the seller point is more than 95 or not.

def recom(x,y):
    if y>=400 or x<95:
        return 'NOT RECOMMEND'
    else:
        return 'RECOMMENDED'

In [616]:
# I applied my function and wrote them in the dataframe.

df['recom']=list(map(recom,df['satici_pts'],df['fark']))
df=df.drop('fark',axis=1)

In [617]:
# Sort DataFrame by discount rate descending.

df=df.sort_values(by=['indirim_orani'],ascending=False)
df=df.reset_index(drop=True)
df.drop('urun_eski_fiyat',axis=1)

,urun_ismi,urun_yeni_fiyat,urun_satici,satici_pts,indirim_orani,recom
0,Apple iPhone 6S 32 GB (Apple Türkiye Garantili),4199.0,akay-bilişim,100.0,19.0,RECOMMENDED
1,Apple iPhone 6S 32 GB (Apple Türkiye Garantili),3999.9,Justfather,95.0,18.0,RECOMMENDED
2,Apple iPhone 6S 16 GB (Apple Türkiye Garantili),3099.0,Azur,95.0,13.0,RECOMMENDED
3,Apple iPhone 6S 32 GB (Apple Türkiye Garantili),3850.0,ÖZDEMİRİLETİŞİMM,0.0,4.0,NOT RECOMMEND
4,Apple iPhone 6S Plus 32 GB (Apple Türkiye Gara...,5398.0,akay-bilişim,100.0,2.0,NOT RECOMMEND
5,Apple iPhone 6S 32 GB (Apple Türkiye Garantili),3750.0,Yenigünİletişim,99.0,0.0,RECOMMENDED
6,Apple iPhone 6S 32 GB (Apple Türkiye Garantili),3499.0,İdeaLMobiL,100.0,0.0,RECOMMENDED
7,Apple iPhone 6S 64 GB (İthalatçı Garantili),2999.0,Elektronikadam,47.0,0.0,NOT RECOMMEND
8,Apple iPhone 6S 16 GB (İthalatçı Garantili),1699.0,starmobile,100.0,0.0,NOT RECOMMEND


In [618]:
# To write csv file.
df.to_csv('kronnika.csv')

##### Thanks for your interest...